In [2]:
# import os

# os.chdir("..")
!ls
# !pwd
# !rm -rf input

logs  output  proper_test_dataset.csv  semantic_fine_tuned2


In [3]:
import numpy; print(numpy.__version__)

1.26.4


In [4]:
# %pip install datasets==2.18.0
# !pip install --upgrade pandas
# %pip install evaluate
# !pip install pandas==1.4.3
!pip install evaluate
# !pip install numpy==1.19.5
# !pip install accelerate
# !pip install --upgrade torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00


In [6]:
import pandas; print(pandas.__version__)

2.2.1


In [7]:
from transformers import RobertaConfig, RobertaTokenizerFast,RobertaForSequenceClassification, Trainer, TrainingArguments, RobertaTokenizer
from tokenizers.processors import TemplateProcessing
import torch
from torch.utils.data import Dataset
from datasets import Dataset, load_dataset
import os
from pathlib import Path
import numpy as np
import evaluate
import accelerate
import pandas as pd

dataset_path = Path("/kaggle/input/semanticbenchmark-dataset/semantic_benchmark_dataset_2.csv")
print(dataset_path.exists())

df = pd.read_csv(dataset_path)

# Convert the Pandas DataFrame into a datasets Dataset
dataset = Dataset.from_pandas(df)


# dataset = load_dataset('csv', data_files=str(dataset_path), split="train")

2024-04-09 10:03:39.510724: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 10:03:39.510856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 10:03:39.646718: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


True


In [8]:
df["semantic_clone"].value_counts()

semantic_clone
1    1000
0    1000
Name: count, dtype: int64

In [9]:
model_name = "microsoft/codebert-base"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
# tokenizer = RobertaTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

In [10]:
dataset = dataset.rename_column("semantic_clone", "label")

In [ ]:
514/2

In [ ]:
511 / 2

In [11]:
def tokenization(row): 
    tokenized_inputs = tokenizer([row["clone1"], row["clone2"]], padding="max_length", truncation=True, return_tensors="pt",
                                 max_length=255)
    tokenized_inputs["input_ids"] = tokenized_inputs["input_ids"].flatten()
    tokenized_inputs["attention_mask"] = tokenized_inputs["attention_mask"].flatten()
#     print(tokenized_inputs.shape)
    return tokenized_inputs

dataset = dataset.map(tokenization, batched=False)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
dataset

In [12]:
dataset = dataset.shuffle(seed=42)
dataset_train = dataset.select(range(1200))
proper_test_dataset = dataset.select(range(1200,2000))
proper_test_dataset.to_csv("proper_test_dataset.csv")

dataset_train = dataset_train.train_test_split(test_size=0.16, seed=42)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
dataset_train

In [13]:
np.object = np.object_
dataset_train["train"]["input_ids"]

tensor([[    0,  9232,  5448,  ..., 50117, 50117,     2],
        [    0,  9232,  5448,  ...,     1,     1,     1],
        [    0,  9232,  5448,  ..., 50118, 50117,     2],
        ...,
        [    0,  9232,  5448,  ...,     1,     1,     1],
        [    0,  9232,  5448,  ...,     1,     1,     1],
        [    0,  9232,  5448,  ...,  4832, 50118,     2]])

In [14]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaForSequenceClassification.from_pretrained(model_name, config=config).to(device)

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
config.max_position_embeddings

In [ ]:
# dataset_train["train"][0]["attention_mask"].shape
for sample in dataset:
    if sample["attention_mask"].shape != torch.Size([514]) or sample["input_ids"].shape != torch.Size([514]):
      print(f"Unexpected shape found: {sample}")


In [ ]:
model.roberta.embeddings.word_embeddings

In [ ]:
unique_ids = set()
for sample in dataset:
  unique_ids.update(set(sample["input_ids"].tolist()))  # Convert input_ids to a list and add unique IDs to the set


In [ ]:
word_embeddings = model.roberta.embeddings.word_embeddings
for unique_id in unique_ids:
  x = word_embeddings(torch.tensor([unique_id]))
  if x == None :
        print(unique_id)

max(unique_ids)

In [ ]:
for unique_id in unique_ids:
    print(tokenizer.create_position_ids_from_input_ids(unique_id))
    break

In [ ]:
config

In [ ]:
batch_input_ids = dataset_train["train"]["input_ids"][0:2]
batch_attention_mask = dataset_train["train"]["attention_mask"][0:2]

model(input_ids=batch_input_ids.to(device), attention_mask=batch_attention_mask.to(device))


In [ ]:
# from torch.utils.data import DataLoader
from transformers import EarlyStoppingCallback, IntervalStrategy

accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")
# train_dataloader = DataLoader(dataset_train["train"], batch_size=2, shuffle=True)
# eval_dataloader = DataLoader(dataset_train["test"], batch_size=2)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"], "precision": precision.compute(predictions=predictions, references=labels)["precision"], "recall": recall.compute(predictions=predictions, references=labels)["recall"], "f1": f1.compute(predictions=predictions, references=labels)["f1"]}

# optimizer: AdamW default
training_args = TrainingArguments(
    per_device_train_batch_size=16,  # Batch size per GPU/TPU core/CPU for training
    per_device_eval_batch_size=16,
    learning_rate=2e-5,             # Learning rate
    adam_epsilon=1e-8,              # Epsilon for Adam optimizer
    num_train_epochs=30,             # Total number of training epochs
    # block_size=400,                 # Block size
    logging_dir='./logs',           # Directory for storing logs
    logging_steps=16,
    evaluation_strategy="steps",
    eval_steps=16,
    output_dir ="./output",
    dataloader_pin_memory=True,
    dataloader_num_workers=4,
    report_to=None,
    do_eval=True,                 # Perform evaluation at the end of training
    save_strategy="steps",
    save_steps=16,
    load_best_model_at_end = True,
    metric_for_best_model = 'f1',
    save_total_limit=2
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train["train"],
    eval_dataset=dataset_train["test"],      # Evaluation dataset
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=4)],
)

# Train the model
trainer.train()
# trainer.train(resume_from_checkpoint=True)

In [19]:
# trainer.train(resume_from_checkpoint=True)
# trainer.evaluate()
# trainer.save_model(f"semantic_fine_tuned2")
trainer.evaluate(proper_test_dataset)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

{'eval_loss': 0.30004990100860596,
 'eval_accuracy': 0.9125,
 'eval_precision': 0.8883610451306413,
 'eval_recall': 0.9420654911838791,
 'eval_f1': 0.9144254278728605,
 'eval_runtime': 14.7417,
 'eval_samples_per_second': 54.268,
 'eval_steps_per_second': 1.696,
 'epoch': 8.5}

In [ ]:
torch.cuda.empty_cache()

import gc

gc.collect()

In [ ]:
# del model
# del trainer
# torch.cuda.empty_cache()

In [21]:
!zip -r /kaggle/working/best-model.zip /kaggle/working/semantic_fine_tuned2/*


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/semantic_fine_tuned2/config.json (deflated 50%)
  adding: kaggle/working/semantic_fine_tuned2/model.safetensors (deflated 7%)
  adding: kaggle/working/semantic_fine_tuned2/training_args.bin (deflated 51%)
